In [95]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier

In [96]:
train_data = pd.read_csv('./train.csv')
train_ans = pd.read_csv('./train_answers.csv')

In [97]:
train_ans = train_ans.iloc[:,1]
y = np.repeat(train_ans.to_numpy(), 4)

Tortuosity features

In [98]:
train_tort_data = train_data.iloc[:, 4:44]

x = train_tort_data.to_numpy()

In [99]:
print(np.shape(x))
print(np.shape(y))

print(f'\nThere are {(y==0).sum()} male writers.')
print(f'\nThere are {(y==1).sum()} male writers.')

(1128, 40)
(1128,)

There are 572 male writers.

There are 556 male writers.


In [103]:
model = SGDClassifier()
model.fit(X=x, y=y)

SGDClassifier()

In [104]:
answers = model.predict(X=x)

In [105]:
print((answers != y).sum())
print(answers[:32])
print(y[:32])

439
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Curve features

In [106]:
train_curve_data = train_data.iloc[:, 54:900]
x = train_curve_data.to_numpy()

In [107]:
print(np.shape(x))
print(np.shape(y))

(1128, 846)
(1128,)


In [108]:
model.fit(X=x, y=y)

SGDClassifier()

In [109]:
answers = model.predict(X=x)

In [110]:
print((answers != y).sum())
print(answers[:32])
print(y[:32])

499
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Chaincode features?

In [111]:
train_chain_data = train_data.iloc[:, 901:5020]
x = train_chain_data.to_numpy()

In [112]:
print(np.shape(x))
print(np.shape(y))

(1128, 4119)
(1128,)


In [113]:
model.fit(X=x, y=y)

SGDClassifier()

In [114]:
answers = model.predict(X=x)

In [115]:
print((answers != y).sum())
print(answers[:32])
print(y[:32])

516
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


Try cross-validation using chaincode features

In [116]:
chaincode_features = (train_data.iloc[:, 901:5020]).to_numpy()
test_idx = np.arange(3, 1128, 4)
train_idx = np.delete(np.arange(1128), test_idx)

x_test = chaincode_features[test_idx, :]
x_train = chaincode_features[train_idx, :]

In [117]:
print(np.shape(x_test))
print(np.shape(x_train))

(282, 4119)
(846, 4119)


In [118]:
y_train = np.repeat(train_ans.to_numpy(), 3)
y_test = train_ans

In [119]:
model.fit(X=x_train, y=y_train)

SGDClassifier()

In [120]:
print(model.score(X=x_train, y=y_train))
print((model.predict(X=x_train) != y_train).sum())
print('\n')
print(model.score(X=x_test, y=y_test))
print((model.predict(X=x_test) != y_test).sum())

0.516548463356974
409


0.5283687943262412
133
